# Modello euristico con gruppi pre-aggregati

## Import pacchetti

In [ ]:
# ------------------------------------------------------------
# 0) IMPORT
# ------------------------------------------------------------
from tabnanny import verbose
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor, as_completed
import pickle

# Notebook esterno con tutte le funzioni di routing
import import_ipynb
import performance_calc as pc

## Caricamento dati pre-aggregati

In [ ]:
# Carica il dizionario dei gruppi pre-aggregati
with open('clusters_output_punti_simili_full.pkl', 'rb') as f:
    aggregated_groups = pickle.load(f)

print(f"Gruppi caricati: {len(aggregated_groups)}")
print(f"Esempio gruppo: {list(aggregated_groups.items())[0]}")

## Funzioni di supporto per gestire i gruppi

In [ ]:
def prepare_aggregated_data(aggregated_groups, delivery_points):
    """
    Prepara i dati per il clustering considerando i gruppi aggregati.
    
    Args:
        aggregated_groups: dizionario {chiave: (centroid_id, [location_ids])}
        delivery_points: DataFrame originale con tutti i punti
    
    Returns:
        aggregated_points_df: DataFrame con i centroidi dei gruppi
        group_mapping: dizionario {centroid_id: [location_ids nel gruppo]}
        group_sizes: dizionario {centroid_id: numero di punti nel gruppo}
    """
    group_mapping = {}
    group_sizes = {}
    centroid_ids = []
    
    for key, (centroid_id, location_ids) in aggregated_groups.items():
        group_mapping[centroid_id] = location_ids
        group_sizes[centroid_id] = len(location_ids)
        centroid_ids.append(centroid_id)
    
    # Crea DataFrame con solo i centroidi
    aggregated_points_df = delivery_points[
        delivery_points['location_id'].isin(centroid_ids)
    ].copy()
    
    print(f"Gruppi totali: {len(group_mapping)}")
    print(f"Punti totali nei gruppi: {sum(group_sizes.values())}")
    print(f"Punti medi per gruppo: {sum(group_sizes.values()) / len(group_sizes):.2f}")
    
    return aggregated_points_df, group_mapping, group_sizes


def expand_cluster_with_groups(cluster_centroid_ids, group_mapping):
    """
    Espande un cluster da centroidi a tutti i location_id reali.
    
    Args:
        cluster_centroid_ids: lista di centroid_id
        group_mapping: dizionario {centroid_id: [location_ids]}
    
    Returns:
        lista di tutti i location_id espansi
    """
    expanded_cluster = []
    for centroid_id in cluster_centroid_ids:
        expanded_cluster.extend(group_mapping[centroid_id])
    return expanded_cluster


def calculate_cluster_unloading_time(cluster_centroid_ids, group_sizes, unloading_time_per_point=10):
    """
    Calcola il tempo totale di scarico considerando tutti i punti nei gruppi.
    
    Args:
        cluster_centroid_ids: lista di centroid_id nel cluster
        group_sizes: dizionario {centroid_id: numero di punti}
        unloading_time_per_point: minuti per punto (default 10)
    
    Returns:
        tempo totale di scarico in minuti
    """
    total_points = sum(group_sizes[cid] for cid in cluster_centroid_ids)
    return total_points * unloading_time_per_point

## Algoritmo di clustering adattato

In [ ]:
def clustering_euristico_multicluster_aggregated(full_time_mat_min: np.ndarray,
                                                  index_to_location_id: list[int],
                                                  aggregated_points: pd.DataFrame,
                                                  group_sizes: dict,
                                                  max_time: int = 500,
                                                  unloading_time_per_point: int = 10):
    """
    Algoritmo Cluster-First, Route-Second adattato per gruppi pre-aggregati.
    
    • seed = gruppo più distante dal deposito
    • greedy: aggiunge il gruppo che allunga meno il tour
    • stop quando supererebbe max_time
    • Il tempo di unloading considera TUTTI i punti nel gruppo (non solo il centroide)
    """
    n_groups = full_time_mat_min.shape[0]
    unassigned = set(range(1, n_groups))  # deposito è indice 0
    clusters = []
    
    while unassigned:
        # --- nuovo cluster
        first_idx = max(unassigned, key=lambda i: full_time_mat_min[0, i])
        cluster = [first_idx]
        unassigned.remove(first_idx)
        
        # IMPORTANTE: il tempo di unloading considera TUTTI i punti nel gruppo
        first_centroid_id = index_to_location_id[first_idx]
        first_group_points = group_sizes[first_centroid_id]
        cluster_time = full_time_mat_min[0, first_idx] * 2 + (first_group_points * unloading_time_per_point)
        
        # --- inserimento greedy
        while unassigned:
            # tempo minimo dal cluster al candidato
            cand, min_cost = min(
                ((c, min(full_time_mat_min[p, c] for p in cluster)) for c in unassigned),
                key=lambda x: x[1]
            )
            
            # IMPORTANTE: calcola unloading time per tutti i punti del gruppo candidato
            cand_centroid_id = index_to_location_id[cand]
            cand_group_points = group_sizes[cand_centroid_id]
            cand_unloading_time = cand_group_points * unloading_time_per_point
            
            if cluster_time + min_cost + cand_unloading_time > max_time:
                break
            
            cluster.append(cand)
            unassigned.remove(cand)
            cluster_time += min_cost + cand_unloading_time
        
        # Converti indici in centroid_ids
        clusters.append([index_to_location_id[i] for i in cluster])
    
    return clusters

In [ ]:
def plot_clusters(clusters, delivery_points, title):
    """Visual plot rapido"""
    cmap = plt.cm.get_cmap("tab20", len(clusters))
    plt.figure(figsize=(10, 7))
    for i, cl in enumerate(clusters):
        sub = delivery_points[delivery_points["location_id"].isin(cl)]
        plt.scatter(sub["lon"], sub["lat"], c=[cmap(i)],
                   label=f"C{i+1}", alpha=.7, s=20, edgecolors="k")
    plt.title(title)
    plt.xlabel("lon"); plt.ylabel("lat")
    plt.legend(ncol=2, fontsize="small")
    plt.show()


def deduplicate_clusters(clusters, already_assigned_groups):
    """
    Rimuove gruppi già assegnati dai cluster e restituisce solo cluster validi
    
    Args:
        clusters: lista di cluster (liste di centroid_id)
        already_assigned_groups: set di centroid_id già assegnati
    
    Returns:
        result: lista di cluster puliti
        assigned: set aggiornato di gruppi assegnati
    """
    assigned = set(already_assigned_groups)
    result = []
    
    for cluster in clusters:
        clean_cluster = [cid for cid in cluster if cid not in assigned]
        if clean_cluster:  # Solo se il cluster ha almeno un gruppo
            result.append(clean_cluster)
            assigned.update(clean_cluster)
    
    return result, assigned


def get_last_iteration_rejected_clusters(all_results, already_assigned_groups, verbose=True):
    """
    Recupera i cluster RIFIUTATI dell'ULTIMA iterazione, filtrando i gruppi già assegnati
    
    Args:
        all_results: dizionario con i risultati di tutte le iterazioni
        already_assigned_groups: set di centroid_id già assegnati
        verbose: stampa messaggi di debug
    
    Returns:
        lista di cluster filtrati
    """
    if not all_results:
        return []
    
    # Prendi l'ultima iterazione
    last_iteration = max(all_results.keys())
    last_results = all_results[last_iteration]
    
    # Recupera i cluster rifiutati
    rejected_cluster_indices = last_results['rejected_indices']
    all_clusters = last_results['clusters']
    rejected_clusters = [all_clusters[i] for i in rejected_cluster_indices]
    
    if verbose:
        print(f" 🔍 Recuperando cluster rifiutati dall'ultima iterazione ({last_iteration})")
        print(f" 📋 Cluster rifiutati disponibili: {len(rejected_clusters)}")
    
    # Filtra gruppi già assegnati
    filtered_clusters, _ = deduplicate_clusters(rejected_clusters, already_assigned_groups)
    
    if verbose:
        total_groups = sum(len(cl) for cl in filtered_clusters)
        print(f" ✅ Cluster recuperati dopo filtro: {len(filtered_clusters)} ({total_groups} gruppi)")
    
    return filtered_clusters


def count_clusters_with_excessive_overtime(performance_df, threshold=1):
    """
    Conta quanti cluster hanno più di 'threshold' giorni di overtime
    
    Args:
        performance_df: DataFrame con le performance dei cluster
        threshold: soglia di overtime (default 1)
    
    Returns:
        numero di cluster con overtime > threshold
    """
    count = 0
    for cluster_name, cluster_data in performance_df.groupby('cluster'):
        total_overtime = cluster_data['n_overtime_days'].sum()
        if total_overtime > threshold:
            count += 1
    return count


def evaluate_and_accept_clusters(clusters, performance_df, verbose=True):
    """
    Valuta i cluster e decide quali accettare basandosi sui criteri:
    1) Media giornaliera tra 390-480 minuti
    2) Overtime accettabile (massimo 1 giorno per tipo di giorno della settimana)
    
    Returns:
        accepted_clusters: list di cluster (liste di centroid_id) accettati
        rejected_indices: list di indici dei cluster rifiutati
    """
    accepted_clusters = []
    rejected_indices = []
    
    # Raggruppa performance per cluster
    for cluster_name, cluster_data in performance_df.groupby('cluster'):
        cluster_index = int(cluster_name.split()[-1]) - 1  # "Cluster 1" -> index 0
        cluster = clusters[cluster_index]
        
        # CRITERIO 1: Media giornaliera tra 390-480 minuti
        mean_minutes_range = cluster_data['mean_minutes'].between(390, 480).any()
        
        # CRITERIO 2: Overtime accettabile
        overtime_acceptable = check_overtime_acceptable(cluster_data)
        
        if mean_minutes_range or overtime_acceptable:
            accepted_clusters.append(cluster)
            if verbose:
                max_mean = cluster_data['mean_minutes'].max()
                total_overtime = cluster_data['n_overtime_days'].sum()
                reason = "mean_range" if mean_minutes_range else "overtime_ok"
                print(f" ✅ {cluster_name}: {len(cluster)} gruppi, "
                     f"max_mean={max_mean:.1f}min, overtime={total_overtime}, "
                     f"motivo={reason}")
        else:
            rejected_indices.append(cluster_index)
            if verbose:
                max_mean = cluster_data['mean_minutes'].max()
                total_overtime = cluster_data['n_overtime_days'].sum()
                print(f" ❌ {cluster_name}: {len(cluster)} gruppi, "
                     f"max_mean={max_mean:.1f}min, overtime={total_overtime}, "
                     f"RIFIUTATO")
    
    return accepted_clusters, rejected_indices


def check_overtime_acceptable(cluster_data):
    """
    Controlla se l'overtime è accettabile:
    massimo 1 giorno di overtime per ogni giorno della settimana
    
    Args:
        cluster_data: DataFrame con performance di un singolo cluster
    
    Returns:
        bool: True se overtime accettabile, False altrimenti
    """
    # Controlla per ogni giorno della settimana
    for _, row in cluster_data.iterrows():
        if row['n_overtime_days'] > 1:  # Più di 1 giorno di overtime -> non accettato
            return False
        elif row['n_overtime_days'] == 1:
            return True  # Ok, 1 giorno di overtime è accettabile
    
    return False

## Funzione principale iterativa

In [ ]:
def heuristic_iterative_clustering_aggregated(aggregated_groups: dict,
                                              delivery_points: pd.DataFrame,
                                              depot_location: tuple[float, float],
                                              initial_threshold: int = 800,
                                              increment_threshold: int = 120,
                                              unload_min_per_point: int = 10,
                                              max_iterations: int = 10,
                                              verbose: bool = True):
    """
    Algoritmo iterativo per clustering con gruppi pre-aggregati.
    
    Args:
        aggregated_groups: dizionario {chiave: (centroid_id, [location_ids])}
        delivery_points: DataFrame con TUTTI i punti originali
        depot_location: tuple (lat, lon) del deposito
        initial_threshold: soglia iniziale in minuti
        increment_threshold: incremento della soglia per iterazione
        unload_min_per_point: minuti di scarico per punto (default 10)
        max_iterations: numero massimo di iterazioni
        verbose: stampa messaggi di debug
    
    Returns:
        all_results: dict con risultati intermedi
        final_clusters_expanded: lista di cluster con TUTTI i location_id espansi
        final_performance: DataFrame con performance calcolate sui punti reali
        refinement_applied: bool indicante se è stato applicato refinement
    """
    
    if verbose:
        print("🔄 INIZIO clustering iterativo con gruppi pre-aggregati")
        print(f" • Soglia iniziale: {initial_threshold} min")
        print(f" • Incremento: +{increment_threshold} min per iterazione")
    
    # Prepara i dati aggregati
    aggregated_points_df, group_mapping, group_sizes = prepare_aggregated_data(
        aggregated_groups, delivery_points
    )
    
    all_results = {}
    remaining_groups = aggregated_points_df.copy()
    original_groups = set(aggregated_points_df['location_id'])
    already_assigned_groups = set()
    current_threshold = initial_threshold
    
    for iteration in range(max_iterations):
        if verbose:
            print(f"\n🔄 Iterazione {iteration + 1}: soglia = {current_threshold} min")
            print(f" • Gruppi rimanenti: {len(remaining_groups)}")
        
        if remaining_groups.empty:
            if verbose:
                print(" ✅ Nessun gruppo rimanente - STOP")
            break
        
        # ================== CLUSTERING sui centroidi ==================
        dm, tm, idx2loc, _ = pc.distance_matrix_creation('', remaining_groups)
        
        # Crea dizionario group_sizes solo per i gruppi rimanenti
        remaining_group_sizes = {
            cid: group_sizes[cid] for cid in remaining_groups['location_id']
        }
        
        clusters_centroids = clustering_euristico_multicluster_aggregated(
            tm, idx2loc, remaining_groups,
            remaining_group_sizes,
            max_time=current_threshold,
            unloading_time_per_point=unload_min_per_point
        )
        
        if not clusters_centroids:
            if verbose:
                print(" ⚠️ Nessun cluster generato - STOP")
            break
        
        # ================== ESPANDI cluster per routing e performance ==================
        # Per il calcolo delle performance, espandi i centroidi in punti reali
        clusters_expanded = [
            expand_cluster_with_groups(cluster, group_mapping)
            for cluster in clusters_centroids
        ]
        
        if verbose:
            print(f" 📦 Cluster creati: {len(clusters_expanded)}")
            # for i, cl in enumerate(clusters_expanded):
            #     print(f"    Cluster {i+1}: {len(cl)} punti reali")
        
        # ================== PERFORMANCE sui punti reali ==================
        performance_df = pc.calc_clusters_stats(
            clusters_expanded, time_limit=3, parallel=True, max_workers=7, verbose=False
        )
        
        # ================== VALUTAZIONE E ACCETTAZIONE ==================
        # Valuta usando i centroidi, ma le performance sono sui punti reali
        accepted_clusters_centroids, rejected_cluster_indices = evaluate_and_accept_clusters(
            clusters_centroids, performance_df, verbose=verbose
        )
        
        # Salva risultati dell'iterazione
        all_results[iteration + 1] = {
            'threshold': current_threshold,
            'clusters': clusters_centroids,  # Salva centroidi
            'clusters_expanded': clusters_expanded,  # Salva anche espansi
            'performance_df': performance_df,
            'accepted_clusters': accepted_clusters_centroids,
            'rejected_indices': rejected_cluster_indices
        }
        
        if verbose:
            print(f" ✅ Cluster accettati: {len(accepted_clusters_centroids)}/{len(clusters_centroids)}")
        
        # ================== RIMOZIONE GRUPPI ACCETTATI ==================
        accepted_group_ids = set()
        for cluster in accepted_clusters_centroids:
            accepted_group_ids.update(cluster)
        
        existing_ids = set(remaining_groups['location_id'])
        valid_accepted_ids = accepted_group_ids.intersection(existing_ids)
        
        already_assigned_groups.update(valid_accepted_ids)
        remaining_groups = remaining_groups[
            ~remaining_groups['location_id'].isin(valid_accepted_ids)
        ].copy()
        
        # Se tutti i cluster sono stati accettati, abbiamo finito
        if len(accepted_clusters_centroids) == len(clusters_centroids):
            if verbose:
                print(" 🎯 Tutti i cluster sono stati accettati - STOP")
            break
        
        # Incrementa soglia per prossima iterazione
        current_threshold += increment_threshold
    
    # ================== ASSEMBLAGGIO FINALE ==================
    if verbose:
        print(f"\n🔧 Assemblaggio soluzione finale...")
    
    final_clusters_centroids = []
    
    # 1. Aggiungi tutti i cluster accettati
    for iteration_results in all_results.values():
        for cluster in iteration_results['accepted_clusters']:
            final_clusters_centroids.append(cluster)
    
    # 2. Per i gruppi rimanenti, usa i cluster rifiutati dell'ultima iterazione
    unassigned_groups = original_groups - already_assigned_groups
    
    if unassigned_groups:
        if verbose:
            print(f" 📋 Gruppi non assegnati: {len(unassigned_groups)}")
            print(" 🔍 Recuperando cluster rifiutati dall'ultima iterazione...")
        
        last_iteration_clusters = get_last_iteration_rejected_clusters(
            all_results, already_assigned_groups, verbose
        )
        
        final_clusters_centroids.extend(last_iteration_clusters)
        for cluster in last_iteration_clusters:
            already_assigned_groups.update(cluster)
    
    # 3. ESPANDI TUTTI i cluster finali da centroidi a punti reali
    final_clusters_expanded = [
        expand_cluster_with_groups(cluster, group_mapping)
        for cluster in final_clusters_centroids
    ]
    
    # ================== PERFORMANCE FINALI sui punti reali ==================
    if verbose:
        print(f"\n📊 Calcolando performance finali per {len(final_clusters_expanded)} cluster...")
    
    final_performance = pc.calc_clusters_stats(
        final_clusters_expanded, time_limit=3, parallel=True, max_workers=7, verbose=False
    )
    
    if verbose:
        total_groups = sum(len(cluster) for cluster in final_clusters_centroids)
        total_points = sum(len(cluster) for cluster in final_clusters_expanded)
        print(f"\n🏁 COMPLETATO:")
        print(f" • Iterazioni eseguite: {len(all_results)}")
        print(f" • Cluster finali totali: {len(final_clusters_expanded)}")
        print(f" • Gruppi totali assegnati: {total_groups}")
        print(f" • Punti totali assegnati: {total_points}")
        print(f" • Copertura: {total_points}/{len(delivery_points)} punti")
    
    # ================== CHECK OVERTIME ECCESSIVO E REFINEMENT ==================
    excessive_overtime_count = count_clusters_with_excessive_overtime(final_performance, threshold=1)
    
    if verbose:
        print(f"\n📊 Cluster con più di 1 giorno overtime: {excessive_overtime_count}")
    
    refinement_needed = excessive_overtime_count > 5
    
    if refinement_needed:
        if verbose:
            print(f"\n🔄 REFINEMENT NECESSARIO: {excessive_overtime_count} cluster con overtime > 1")
            print(" 🚀 Avvio 5 cicli aggiuntivi con soglia 1400 min (+60/iterazione)...")
        
        # Identifica i gruppi nei cluster problematici
        problematic_groups = set()
        for cluster_name, cluster_data in final_performance.groupby('cluster'):
            total_overtime = cluster_data['n_overtime_days'].sum()
            if total_overtime > 1:
                cluster_index = int(cluster_name.split()[-1]) - 1
                problematic_cluster_centroids = final_clusters_centroids[cluster_index]
                problematic_groups.update(problematic_cluster_centroids)
        
        # Crea DataFrame con i gruppi problematici
        refinement_groups_df = aggregated_points_df[
            aggregated_points_df['location_id'].isin(problematic_groups)
        ].copy()
        
        # Crea dizionario aggregated_groups solo per i gruppi problematici
        refinement_aggregated_groups = {
            k: v for k, v in aggregated_groups.items()
            if v[0] in problematic_groups
        }
        
        if verbose:
            print(f" 📋 Gruppi da riclusterare: {len(refinement_groups_df)}")
        
        # Applica refinement (ricorsivo)
        refinement_results, refinement_clusters, refinement_performance, _ = heuristic_iterative_clustering_aggregated(
            aggregated_groups=refinement_aggregated_groups,
            delivery_points=delivery_points,
            depot_location=depot_location,
            initial_threshold=1400,
            increment_threshold=60,
            unload_min_per_point=unload_min_per_point,
            max_iterations=5,
            verbose=verbose
        )
        
        # Sostituisci i cluster problematici con quelli raffinati
        problematic_points_expanded = set()
        for gid in problematic_groups:
            problematic_points_expanded.update(group_mapping[gid])
        
        final_clusters_expanded = [
            cl for cl in final_clusters_expanded
            if not any(p in problematic_points_expanded for p in cl)
        ]
        final_clusters_expanded.extend(refinement_clusters)
        
        # Ricalcola performance finali
        final_performance = pc.calc_clusters_stats(
            final_clusters_expanded, time_limit=3, parallel=True, max_workers=7, verbose=False
        )
        
        if verbose:
            print(f"\n✅ REFINEMENT COMPLETATO")
            print(f" • Nuovi cluster totali: {len(final_clusters_expanded)}")
            excessive_after = count_clusters_with_excessive_overtime(final_performance, threshold=1)
            print(f" • Cluster con overtime > 1: {excessive_after} (prima: {excessive_overtime_count})")
        
        return all_results, final_clusters_expanded, final_performance, True
    
    return all_results, final_clusters_expanded, final_performance, False

## Esecuzione

In [ ]:
# Esegui il clustering
all_results, final_clusters, final_performance, refinement_applied = heuristic_iterative_clustering_aggregated(
    aggregated_groups=aggregated_groups,
    delivery_points=pc.delivery_points,
    depot_location=pc.depot_location,
    initial_threshold=800,
    increment_threshold=100,
    unload_min_per_point=10,
    max_iterations=15,
    verbose=True
)

# Stampa risultato refinement
if refinement_applied:
    print("\n🎯 REFINEMENT APPLICATO con successo!")
else:
    print("\n✅ NESSUN REFINEMENT necessario - soluzione ottimale trovata")

## Salvataggio risultati

In [ ]:
# Salva i risultati
final_performance.to_csv('clustering_methods_performances/euristic_aggregated.csv')

with open('cluster_dicts/cluster_dict_euristic_aggregated.pkl', 'wb') as f:
    pickle.dump(final_clusters, f)

print("✅ Risultati salvati con successo!")